In [8]:
import pandas as pd
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import numpy as np
from sklearn import metrics

In [9]:
# Import the two datasets 

df = pd.read_csv("log_windowed_labeled.csv", header = 0).fillna(0)
df

,Start time,End time,Hall-Bedroom door,Hall-Bathroom door,ToiletFlush,Plates cupboard,Fridge,Microwave,Groceries Cupboard,Hall-Toilet door,Frontdoor,Cups cupboard,Pans Cupboard,Freezer,Dishwasher,Washingmachine,Label,Label_ID
0,2008-02-25 09:33:00,2008-02-25 09:33:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,go to bed,10
1,2008-02-25 09:33:00,2008-02-25 09:37:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,go to bed,10
2,2008-02-25 09:33:00,2008-02-25 09:37:00,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,use toilet,4
3,2008-02-25 09:33:00,2008-02-25 09:37:00,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,use toilet,4
4,2008-02-25 09:33:00,2008-02-25 09:38:00,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,use toilet,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,2008-03-21 15:53:00,2008-03-21 18:24:00,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,go to bed,10
1078,2008-03-21 17:03:00,2008-03-21 18:24:00,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,go to bed,10
1079,2008-03-21 17:06:00,2008-03-21 18:25:00,3.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,use toilet,4
1080,2008-03-21 17:06:00,2008-03-21 19:11:00,2.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,use toilet,4


In [10]:
data_top = df.columns
columns = list(data_top) 

In [11]:
features = columns
features.remove("Label")
features.remove("Label_ID")
features.remove("Start time")
features.remove("End time")
features

['Hall-Bedroom door',
 'Hall-Bathroom door',
 'ToiletFlush',
 'Plates cupboard',
 'Fridge',
 'Microwave',
 'Groceries Cupboard',
 'Hall-Toilet door',
 'Frontdoor',
 'Cups cupboard',
 'Pans Cupboard',
 'Freezer',
 'Dishwasher',
 'Washingmachine']

In [12]:
X = df[features]
y = df.Label_ID

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.3, random_state=1) # 70% training and 30% test
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
data = tree.export_graphviz(clf, out_file=None, feature_names=features)
graph = pydotplus.graph_from_dot_data(data)
graph.write_png('mydecisiontree_classifier_test.png')

#Predict the response for test dataset
y_pred = clf.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# img=pltimg.imread('mydecisiontree.png')
# imgplot = plt.imshow(img)
# plt.show() 


Accuracy: 0.68


In [14]:
print("Trying to predict the take shower activity, with ID 5, \nwith the following combination 1,0,0,0,0,0,0,3,0,0,0,0,0,0")
pred = clf.predict([[1,0,0,0,0,0,0,3,0,0,0,0,0,0]])
print("Value predicted:", pred)
if(pred == 5): print("Correct prediction") 
else: print("Wrong predicition")

Trying to predict the take shower activity, with ID 5, 
with the following combination 1,0,0,0,0,0,0,3,0,0,0,0,0,0
Value predicted: [10]
Wrong predicition
